# Compound

In `tatva` we intend to define a total energy functional that takes a **flat** array $\mathbf{z}$ with all unknown DOFs and returns a scalar.
However, the actual unknowns in $\mathbf{z}$ are most often **nodal fields** with a specific shape, $e.g$ a displacement field `(n_nodes, 2)`.
Furthermore, the DOF array may include **multiple** fields. For example, a different nodal field like temperature `(n_nodes, 1)`, or a set of Lagrange-Multipliers `(n_constraints, n)`.

!!! note "Why pass a flat array $\mathbf{z}$ instead of the shaped fields to the energy function?"

    Passing a single flat array instead of the shaped fields has the advantage that the derivatives obtained through AD have convenient shapes.

    ```python
    residual_fn = jax.jacrev(energy)  # returns a vector of shape (n_dofs,)
    jacobian_fn = jax.facfwd(residual_fn)  # returns a rank 2 tensor of shape (n_dofs, n_dofs)
    ```

## Problem



In [3]:
import tatva
import matplotlib.pyplot as plt